In [1]:
#!pip install "tensorflow==2.6.0"
#!pip install tinymlgen==0.2

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
tf.__version__

'2.6.0'

In [4]:
### ignore TensorFlow INFO and WARN messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [5]:
model = tf.keras.Sequential()
model.add(layers.Dense(8, input_dim=2, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))

In [6]:
model.compile(
  loss='binary_crossentropy', 
  optimizer=tf.keras.optimizers.SGD(learning_rate=1), 
  metrics=['accuracy'])

In [7]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [8]:
epochs = 500
model.fit(training_data, target_data, epochs=epochs)

Epoch 1/500
1/1 [==============================] - 0s 445ms/step - loss: 0.6944 - accuracy: 0.5000
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6943 - accuracy: 0.5000
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6942 - accuracy: 0.5000
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6941 - accuracy: 0.5000
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6941 - accuracy: 0.5000
Epoch 6/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6940 - accuracy: 0.5000
Epoch 7/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6940 - accuracy: 0.5000
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6939 - accuracy: 0.5000
Epoch 9/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6938 - accuracy: 0.5000
Epoch 10/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6938 - accuracy: 0.5000
Epoch 11/500
1/1 [=========

1/1 [==============================] - 0s 5ms/step - loss: 0.6904 - accuracy: 0.5000
Epoch 85/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6904 - accuracy: 0.5000
Epoch 86/500
1/1 [==============================] - 0s 7ms/step - loss: 0.6903 - accuracy: 0.5000
Epoch 87/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6903 - accuracy: 0.5000
Epoch 88/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6902 - accuracy: 0.5000
Epoch 89/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6901 - accuracy: 0.5000
Epoch 90/500
1/1 [==============================] - ETA: 0s - loss: 0.6901 - accuracy: 0.50 - 0s 5ms/step - loss: 0.6901 - accuracy: 0.5000
Epoch 91/500
1/1 [==============================] - 0s 9ms/step - loss: 0.6900 - accuracy: 0.5000
Epoch 92/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6899 - accuracy: 0.5000
Epoch 93/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6898 - accuracy

1/1 [==============================] - 0s 5ms/step - loss: 0.6740 - accuracy: 0.7500
Epoch 167/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6735 - accuracy: 0.7500
Epoch 168/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6730 - accuracy: 0.7500
Epoch 169/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6725 - accuracy: 0.7500
Epoch 170/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6720 - accuracy: 0.7500
Epoch 171/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6715 - accuracy: 0.7500
Epoch 172/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6709 - accuracy: 0.7500
Epoch 173/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6704 - accuracy: 0.7500
Epoch 174/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6698 - accuracy: 0.7500
Epoch 175/500
1/1 [==============================] - 0s 7ms/step - loss: 0.6692 - accuracy: 0.7500
Epoch 176/500
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 0.5791 - accuracy: 0.7500
Epoch 249/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5772 - accuracy: 0.7500
Epoch 250/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5752 - accuracy: 0.7500
Epoch 251/500
1/1 [==============================] - 0s 4ms/step - loss: 0.5732 - accuracy: 0.7500
Epoch 252/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5712 - accuracy: 0.7500
Epoch 253/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5692 - accuracy: 0.7500
Epoch 254/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5671 - accuracy: 0.7500
Epoch 255/500
1/1 [==============================] - 0s 4ms/step - loss: 0.5651 - accuracy: 0.7500
Epoch 256/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5630 - accuracy: 0.7500
Epoch 257/500
1/1 [==============================] - 0s 4ms/step - loss: 0.5609 - accuracy: 0.7500
Epoch 258/500
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.3807 - accuracy: 1.0000
Epoch 331/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3781 - accuracy: 1.0000
Epoch 332/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3754 - accuracy: 1.0000
Epoch 333/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3728 - accuracy: 1.0000
Epoch 334/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3701 - accuracy: 1.0000
Epoch 335/500
1/1 [==============================] - 0s 2ms/step - loss: 0.3674 - accuracy: 1.0000
Epoch 336/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3648 - accuracy: 1.0000
Epoch 337/500
1/1 [==============================] - 0s 6ms/step - loss: 0.3621 - accuracy: 1.0000
Epoch 338/500
1/1 [==============================] - 0s 9ms/step - loss: 0.3595 - accuracy: 1.0000
Epoch 339/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3568 - accuracy: 1.0000
Epoch 340/500
1/1 [=====

1/1 [==============================] - 0s 6ms/step - loss: 0.1873 - accuracy: 1.0000
Epoch 413/500
1/1 [==============================] - 0s 3ms/step - loss: 0.1855 - accuracy: 1.0000
Epoch 414/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1837 - accuracy: 1.0000
Epoch 415/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1819 - accuracy: 1.0000
Epoch 416/500
1/1 [==============================] - 0s 3ms/step - loss: 0.1801 - accuracy: 1.0000
Epoch 417/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1784 - accuracy: 1.0000
Epoch 418/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1767 - accuracy: 1.0000
Epoch 419/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1749 - accuracy: 1.0000
Epoch 420/500
1/1 [==============================] - 0s 6ms/step - loss: 0.1732 - accuracy: 1.0000
Epoch 421/500
1/1 [==============================] - 0s 6ms/step - loss: 0.1716 - accuracy: 1.0000
Epoch 422/500
1/1 [=====

1/1 [==============================] - 0s 9ms/step - loss: 0.0874 - accuracy: 1.0000
Epoch 495/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0866 - accuracy: 1.0000
Epoch 496/500
1/1 [==============================] - 0s 3ms/step - loss: 0.0859 - accuracy: 1.0000
Epoch 497/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0852 - accuracy: 1.0000
Epoch 498/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0845 - accuracy: 1.0000
Epoch 499/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0838 - accuracy: 1.0000
Epoch 500/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0831 - accuracy: 1.0000


In [9]:
model.predict(training_data)

array([[0.05354139],
       [0.917638  ],
       [0.92684555],
       [0.10668641]], dtype=float32)

## C-Code

In [10]:
from tinymlgen import port
import os

In [11]:
c_code = port(model, optimize=[tf.lite.Optimize.DEFAULT], pretty_print = True)

path = os.getcwd()
#path = os.getcwd() + "/../pico-tflmicro/examples/xor/"
open(path + "/model.cpp", "w").write('#include "model.h"\n' + c_code)

2021-10-04 23:43:03.725231: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/47/1676p97n13l43zssl0h5hkhh0000gn/T/tmp1vte4fx4/assets


2021-10-04 23:43:04.023482: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-04 23:43:04.023558: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.


10854